<a href="https://colab.research.google.com/github/CristopherCano/Proyecto_Python_Procesamiento_de_datos/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Procesamiento de Datos con Python - BEDU - Módulo 3
## Viajes en Taxis y Ubers en la Zona Metropolitana del Valle de México 2016 - 2017
* Eduardo Alán Hernández Villasana
* Cristopher Arvizu Cano
* David Gil Peralta
* José Antonio Aguilar Télez

## Planteamiento del Problema
En los últimos años, desde la llegada de aplicaciones para el transporte privado como Uber o Didi, los gremios taxistas de la Zona Metropolitana del Valle de México han protestado en contra de estas aplicaciones, ellos argumentan una competencia desleal y exigen regulaciones hacia estos transportes. Esto no es de extrañarse ya que, con el aumento de la popularidad de estas aplicaciones de transporte, los taxistas están perdiendo clientes y no pueden o quieren modernizarse para poder competir con los automóviles de Uber o Didi que son más modernos. 

A raíz de este enfrentamiento, han surgido noticias comparando estos dos modelos de transporte privado, sin embargo, muchas de estas noticias están sesgadas por intereses políticos y económicos. Es por esta razón que decidimos analizar datos recopilados tanto en Taxis y Ubers para hacer una comparación entre los dos modelos y aportar nuestros hallazgos estadísticos a la discusión de este problema.

## Preguntas Clave
Durante el desarrollo de este proyecto, se espera responder las siguientes preguntas.

* ¿Qué tipo de transporte pasa menos tiempo en el tráfico?
* ¿Cuál es la velocidad promedio de cada tipo de transporte?
* ¿Qué día de la semana hay mas demanda para cada tipo de transporte?
* ¿En qué tipo de transporte recibe más dinero en promedio por viaje?
* ¿Desde qué demsarcación territorial o municipio cada tipo de transporte tiene mayor demanda?
* ¿Cuál es el destino mas solicitado para cada tipo de transporte?

In [1]:
# Librerías a usar
import pandas as pd
import numpy as np

### Extracción de datos
Se descargará la base de datos desde GitHub.

In [2]:
url = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte_raw.csv"

In [49]:
data_raw = pd.read_csv(url, index_col=0)
data_raw.head(2)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
1,México DF Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,N,120449,12373,242
2,México DF Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,N,14110,1700,461


### Descripción de las Columnas
* **id**: Un identificador único para cada viaje.
* **vendor_id**: El tipo de transporte en que se realizó el viaje.
* **pickup_datetime**: Fecha y Hora en que el viaje inició.
* **dropoff_datetime**: Fecha y Hora en que el viaje finalizó.
* **pickup_longitude**: Longitud en la que el viaje inició.
* **pickup_latitude**: Latitud en la que el viaje inició.
* **dropoff_longitude**: Longitud en la que el viaje finalizó.
* **dropoff_latitude**: Latitud en la que el viaje finalizó.
* **store_and_fwd_flag**: Indica si la información del viaje se almacenó en una memoria y después se envio al servidor por que no tenía conexión a internet o no. Todas las entradas de datos estan en 'N'.
* **trip_duration**: Duración del viaje en segundos.
* **dist_meters**: Distancia recorrida en el viaje en metros.
* **wait_sec**: Tiempo en segundos en el que el automovil estuvo completamente detenido durante el viaje. (Se usará como medición del tráfico.)

## Exploración básica de datos
### Hallazgos preliminares encontraros 

1. ¿El conjunto de datos que tengo realmente me sirve para responder algunas de las preguntas que me planteé?

In [50]:
tamaño = data_raw.shape
print(f'2. ¿Qué tamaño tiene mi conjunto de datos? \n número de filas: {tamaño[0]} \n número de columnas: {tamaño[1]}')

2. ¿Qué tamaño tiene mi conjunto de datos? 
 número de filas: 12694 
 número de columnas: 11


In [51]:
print(f'3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? \n \n En total tenemos{tamaño[1]} columnas \n \n {data_raw.columns}')

3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? 
 
 En total tenemos11 columnas 
 
 Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'dist_meters', 'wait_sec'],
      dtype='object')


4. Los nombres que tienen mis columnas, ¿son el nombre más apropiado?

Podemos observar que algunos nombres como vendor_id o dist_meters no son los más adecuados por lo que tendremos que usar la conveción apropiada que es snake-case y ajustar sus nombres

5. ¿Qué tipos de datos tengo en cada columna? ¿Parecen ser el tipo correcto de datos? ¿O es un tipo de datos "incorrecto"?



In [54]:
data_raw.dtypes

vendor_id              object
pickup_datetime        object
dropoff_datetime       object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dist_meters             int64
wait_sec                int64
dtype: object

Las fechas aparecen como object por lo que sera necesario transformar esta variable a un formato de fecha mediante padas

6. Si selecciono algunas filas al azar y las observo, ¿estoy obteniendo los datos que debería? ¿O hay datos que parecen estar "sucios" o "incorrectos"?


In [58]:
data_raw.sample(5)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
10013,México DF Taxi de Sitio,2017-06-17 12:54:50,2017-06-17 01:06:10,-99.185879,19.433096,-99.203936,19.402451,N,681,6476,144
2162,México DF Taxi de Sitio,2016-11-26 04:18:12,2016-11-27 01:44:36,-99.179558,19.438655,-99.179473,19.438618,N,77184,2614,321
6102,México DF Taxi Libre,2017-02-23 10:45:49,2017-02-23 11:28:44,-99.236531,19.355038,-99.222888,19.363453,N,2575,2473,2174
11617,México DF Taxi de Sitio,2017-06-18 11:05:49,2017-07-07 09:14:45,-99.113880,19.324233,-99.100779,19.306939,N,1591737,32230,189
2689,México DF Taxi de Sitio,2016-11-02 03:29:32,2016-11-02 03:45:36,-99.179585,19.438574,-99.155744,19.418668,N,964,4858,473


A continuación observamos los siguiente problemas:
* **vendor_id**: Se repite el patrón México DF
* **store_and_fwd_flag**: No es significativa esta variable ya que siempre marca "N" en cada fila
* **trip_duration**: Tenemos duraciones de viaje exageradamente grandes
* **dist_meters**: Tenemos distacias de viaje exageradamente grandes
* **wait_sec**: Tenemos tiempos de espera de viaje exageradamente grandes